# <Section 01 Project: 다음 분기에 어떤 게임을 설계해야 할까?>


In [ ]:
#다음 분기에 출고량이 높을 것이라 예상되는 게임
#게임 설계를 "어떤 플랫폼"을 기반으로 "어떤 장르"로 해야 출고량이 높을지 분석

## 1. 전처리

In [ ]:
import pandas as pd
import numpy as np

def int_to_year(num):
    this_year = 21 #2021년도
    if num < 0 or num == np.nan:
        return num.replace(num, np.nan)
    elif num < this_year:
        return int(num) + 2000
    elif num < 100:
        return int(num) + 1900
    else:
        return int(num)

def convert_str_to_number(string):
    if "K" in string:
        return float(string.replace("K", "")) * (10**3)
    elif "M" in string:
        return float(string.replace("M", "")) * (10**6)
    else:
        return float(string) * (10**6)

#csv파일 읽기
df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv")
df.drop(["Unnamed: 0"], axis = 1, inplace = True)

#데이터에서 결측치가 포함된 row제거 (361개, 전체 데이터의 약 2.71% 에 해당)
df.dropna(axis = 0, inplace = True) #결측치 제거

#출고량 데이터들을 수치형 데이터로 변환
df["Year"] = df["Year"].apply(int_to_year)
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    df[Sales] = df[Sales].apply(convert_str_to_number)

#인덱스 reset
df.reset_index(drop = True, inplace = True)

#2017, 2020년 데이터들은 데이터수가 적어 신뢰성이 부족해 제거함.
del_index = df[(df["Year"] == 2017) | (df["Year"] == 2020)].index
df = df.drop(del_index) 

#확인
df.info()
df

## 2. 분석

### 1. "**지역**"과 "**선호하는 게임장르**"는 연관이 있을것인가?
= 위 물음에 답하기 위해 Chi-square test을 사용할 수 있음.

    (1) 귀무가설: 출고량에 대해 지역과 게임장르는 서로 연관이 없을 것이다. (독립적 O)
    (2) 대안가설: 출고량에 대해 지역과 게임장르는 서로 연관이 있을 것이다. (독립적 X)
    (3) 신뢰도: 95 %

In [ ]:
df_Genre_sales = df.drop(["Name", "Platform", "Publisher"], axis = 1, inplace = False)
df_Genre_sales.head()

In [ ]:
df_Year_Genre_Sales = df_Genre_sales.groupby(["Year", "Genre"])[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]].sum()

df_Year_Genre_Sales.reset_index(drop = False, inplace = True)

df_Year_Genre_Sales.head()

In [ ]:
'''
df_year_sales = df.drop(["Name", "Platform", "Genre", "Publisher"], axis = 1, inplace = False)
Year_Region_sum_list = []
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    Year_sum = df_year_sales[Sales].groupby(df_year_sales["Year"]).sum()
    Year_Region_sum_list.append(Year_sum)

df_Year_Region_sum = pd.concat(Year_Region_sum_list, axis = 1)
df_Year_Region_sum.head()
'''

In [ ]:
#2 sample chi-square test (지역 - 장르의 연관성)
from scipy import stats

p_value_list = []
for i in range(1980, 2017, 1):
    sort = df_Year_Genre_Sales.loc[df_Year_Genre_Sales["Year"] == i]
    sort.index = sort["Genre"]
    sort_drop = sort.drop(["Year", "Genre"], axis = 1, inplace = False)
    for j in sort_drop.columns:
        if sort_drop[j].sum() == 0:
            sort_drop.drop([j], axis = 1, inplace = True)
    chi = stats.chi2_contingency(sort_drop, correction = False)
    pv = chi[1]
    p_value_list.append(pv)

df_p_value = pd.DataFrame(p_value_list, columns = ["p_value"])
df_p_value.index = range(1980, 2017, 1)
df_p_value.head()

In [ ]:
 df_p_value.query("p_value < 0.05").shape[0] / df_p_value.shape[0]
 #모든 연도에서 귀무가설이 기각됨
 #출고량에 대해서 지역과 장르는 연관이 있다고 볼 수 있다.

### 2. 연도에 따른 장르별 시장점유율
- 연도별 게임의 트렌드를 알 수 있다.

In [ ]:
#연도별 게임장르의 트렌드 분석

In [ ]:
#필요없는 column 제거
df_year_sales = df.drop(["Name", "Platform", "Genre", "Publisher"], axis = 1, inplace = False)

#확인
df_year_sales.head()

In [ ]:
#지역별 출고량의 합을 연도별로 구하기
Year_Region_sum_list = []
for Sales in ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]:
    Year_sum = df_year_sales[Sales].groupby(df_year_sales["Year"]).sum()
    Year_Region_sum_list.append(Year_sum)

df_Year_Region_sum = pd.concat(Year_Region_sum_list, axis = 1)
df_Year_Region_sum["All_Sales"] = df_Year_Region_sum.sum(axis = 1)

df_Year_Region_All_Sales = df_Year_Region_sum["All_Sales"]
df_Year_Region_sum.head()

In [ ]:
#지역별 출고량 합
df["All_Sales"] = df["NA_Sales"] + df["EU_Sales"] + df["JP_Sales"] + df["Other_Sales"]

#필요없는 column제거
df_Year_Genre = df.drop(["Name", "Platform", "Publisher", "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"], axis = 1, inplace = False)

#확인
df_Year_Genre.head()

In [ ]:
#연도에 따른 장르별 시장점유율 구하기

#그룹화 (연도-장르, 지역별 출고량 합)
Year_Genre_sum = df_Year_Genre.groupby(["Year", "Genre"])["All_Sales"].sum()

#출고율 구해서 백분율로 변환
Year_Genre_ratio = Year_Genre_sum / df_Year_Region_All_Sales
df_Year_Genre_ratio = pd.DataFrame(Year_Genre_ratio)
df_Year_Genre_ratio = df_Year_Genre_ratio * 100 #백분율

#확인
df_Year_Genre_ratio

In [ ]:
#연도에 따른 장르별 시장점유율 구하기 (시각화)
import matplotlib.pyplot as plt

#색 List
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan", "tan", "slateblue"]

df_Year_Genre_ratio.unstack().plot(kind = 'line',
                                   fontsize = 8,
                                   figsize = (7, 13),
                                   color = colors,
                                   subplots = True,
                                   ylim = [0, 40],
                                   grid = True,
                                   xlabel = "Year",
                                   ylabel = "Share(%)")

plt.show()

### 3. 장르별 출고율에 대한 선형회귀 분석
- (분석을 하다보니 Section 2에서 배울 내용을 포함시켜 PPT에서는 다루지 않았다.)
- 어떤 장르가 단위 게임수(1개) 당 출고량이 많은지를 1차 선형회귀 함수의 기울기를 통해 알 수 있다. 

In [ ]:
#연도-장르별 출고량 합 과 연도-장르별 출시된 게임수

df_Year_Genre_sum = pd.DataFrame(Year_Genre_sum)

Year_Genre_count = df_Year_Genre.groupby(["Year", "Genre"])["Genre"].count()
df_Year_Genre_count = pd.DataFrame(Year_Genre_count)

df_Year_Genre_sum["Count"] = df_Year_Genre_count["Genre"]

#index(연도)를 제거
df_Year_Genre_sum.reset_index(level = 0, drop = True, inplace = True)

#확인
df_Year_Genre_sum

In [ ]:
from sklearn.preprocessing import StandardScaler
import seaborn as sns

labels = df_Year_Genre_sum.index

standard_df_Year_Genre = StandardScaler().fit_transform(df_Year_Genre_sum)
df_standard_Year_Genre = pd.DataFrame(standard_df_Year_Genre, columns = ["All_Sales", "Count"])

df_standard_Year_Genre["Labels"] = labels

#시각화
sns.lmplot(x = 'Count',
           y = 'All_Sales',
           hue = 'Labels',
           #col = 'Labels',
           line_kws={'label':"tip"},
           data = df_standard_Year_Genre,
           #col_wrap = 3,
           order = 1)

In [ ]:
#어떤 장르가 단위 게임수(1개)당 출고량이 높은지를 알 수있다. (기울기가 큰 것일수록 더 높다.)

### 4. 출고량이 높은 게임에 대해 기준 설정 및 전처리

In [ ]:
#K-means clustering을 이용할 것임.
#Elbow method를 이용해 적절한 K값을 결정

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

df_Total_Sales = df[["All_Sales"]]
scaler = StandardScaler()
df_Total_Sales_standard = scaler.fit_transform(df_Total_Sales)

sum_of_squared_distances = []
K = range(1, 15)
for k in K:
    km = KMeans(n_clusters = k)
    km = km.fit(df_Total_Sales_standard)
    sum_of_squared_distances.append(km.inertia_)

plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show() #k값은 6

In [ ]:
#표준화 및 K-means clustering 진행

#지역별 출고량의 합에 대해서 표준화 진행 (Feature 1개)
scaler = StandardScaler()
All_Sales_std = scaler.fit_transform(df[["All_Sales"]])
df_All_Sales_std = pd.DataFrame(All_Sales_std, columns = ["std_All_Sales"])

#K-mean clustering
kmeans = KMeans(n_clusters = 6, random_state = 7)
kmeans.fit(df_All_Sales_std)
K_labels = kmeans.labels_
df_cluster = pd.DataFrame(K_labels, columns = ["Sale_rating"])

df["std_All_Sales"] = df_All_Sales_std["std_All_Sales"]
df["Sale_rating"] = df_cluster["Sale_rating"]

In [ ]:
#K-means clustering 결과 이미지로 구현
plt.rcParams["figure.figsize"] = (7, 7)
#plt.rcParams["figure.figsize"] = (5, 200) #더 자세히 볼 수있음.
sns.scatterplot(x = "Year", y = "std_All_Sales", hue = "Sale_rating", data = df)

In [ ]:
#결과에 따라 출고량이 높다고 판단되는 데이터들 분류
df_Sales_high = df.query("Sale_rating > 0") #상위 14.34 %
df_Sales_low = df.query('Sale_rating == 0') #상위 85.43 %

#출고량이 높다는 기준 정의
rate = (df_Sales_high.shape[0] / (df.shape[0])) * 100
print("'높은 출고량'에 대한 기준을 출고량의 상위", round(rate, 2), "% 로 두었습니다. \n")

#출고량이 높은 데이터 전처리
df_Sales_high.reset_index(drop = True, inplace = True)
df_Sales_high = df_Sales_high.drop(["Sale_rating", "std_All_Sales", "Name", "Publisher"], axis = 1, inplace = False)

#확인
df_Sales_high

### 5. 출고량이 높은 게임에 대한 분석 (플랫폼)

In [ ]:
#출고량이 높은 게임들간의 점유율을 플랫폼별로 나타내기

import seaborn as sns

#출고량이 높은 게임들 간의 연도 - 플랫폼 별로 출시된 게임의 수를 DataFrame화
df_high_Year_Plat_count = pd.crosstab(df_Sales_high.Year, df_Sales_high.Platform, margins=True)

#필요없는 row제거
df_high_Year_Plat_count_drop = df_high_Year_Plat_count.drop(["All"], axis = 0)

#플랫폼별 점유율 계산하기 (출고량이 높은 게임들 사이에서)
for Plat in df_high_Year_Plat_count_drop.columns:
    df_high_Year_Plat_count_drop[Plat] = df_high_Year_Plat_count_drop[Plat] / df_high_Year_Plat_count_drop["All"]

#필요없는 column 제거
df_high_Year_Plat_count_drop2 = df_high_Year_Plat_count_drop.drop(["All"], axis = 1)

#소수점 자릿수 제어
df_high_Year_Plat_ratio = round(df_high_Year_Plat_count_drop2, 2).T

#2010년 이후 데이터만 추림.
df_high_Year_Plat_ratio_drop = df_high_Year_Plat_ratio.drop(df_high_Year_Plat_ratio.columns[:29], axis = 1)

#시각화
plt.rcParams["figure.figsize"] = (15, 10)
sns.heatmap(df_high_Year_Plat_ratio_drop,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.5 , #컬러차트 영역
           )

In [ ]:
#주목하지 않아도 되는 platform제거 (다음분기에 설계할 게임의 플랫폼의 후보에서 제외)
df_high_Year_Plat_ratio_drop2 = df_high_Year_Plat_ratio_drop.drop(["2600", "3DO", "DC", "GB", "GBA", "GC",
                                                                   "GEN", "N64", "NES", "NG", "PS", "PSP",
                                                                   "PSV", "SAT", "SNES", "TG16", "WS", "XB"], axis = 0, inplace = False)

#다시 시각화
plt.rcParams["figure.figsize"] = (15, 10)
sns.heatmap(df_high_Year_Plat_ratio_drop2,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.5 , #컬러차트 영역
           )

plt.show()

In [ ]:
#출고량이 높은 게임이 전체시장에서 차지하는 비율을 플랫폼 별로 나타내기

#출고량이 높은 게임들의 연도-플랫폼별 게임수와 전체 시장에서 연도-플랫폼별 게임수를 DataFrame화
df_high_Year_Plat_count = pd.crosstab(df_Sales_high.Year, df_Sales_high.Platform, margins=True)
df_Year_Plat_count = pd.crosstab(df.Year, df.Platform, margins=True)

#출고량이 높은 게임이 전체시장에서 차지하는 비율을 계산하기
df_high_Year_Plat_ratio_1 = df_high_Year_Plat_count / df_Year_Plat_count

#2010년 이후 데이터만 추림
df_high_Year_Plat_ratio2 = df_high_Year_Plat_ratio_1.T.drop(df_high_Year_Plat_ratio_1.T.columns[:30], axis = 1)

#필요없는 row, column 제거
df_high_Year_Plat_ratio2.drop(["All"], axis = 1, inplace = True)
df_high_Year_Plat_ratio2.drop(["All"], axis = 0, inplace = True)
df_high_Year_Plat_ratio2.drop(["2600", "3DO", "DC", "GB", "GBA", "GC",
                               "GEN", "N64", "NES", "NG", "PS", "PSP",
                               "PSV", "SAT", "SNES", "TG16", "WS", "XB", "GG", "PCFX", "SCD"],axis = 0, inplace = True)

#결측치 처리
df_high_Year_Plat_ratio2.fillna(0)

#시각화
plt.rcParams["figure.figsize"] = (10, 10)
sns.heatmap(df_high_Year_Plat_ratio2,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.4 , #컬러차트 영역
           )

### 6. 출고량이 높은 게임에 대한 분석 (장르)

In [ ]:
#출고량이 높은 게임들간의 점유율을 장르별로 나타내기
import seaborn as sns

#출고량이 높은 게임들 간의 연도 - 장르별로 출시된 게임의 수를 DataFrame화
df_high_Year_Genre_count = pd.crosstab(df_Sales_high.Year, df_Sales_high.Genre, margins=True)

#필요없는 row제거
df_high_Year_Genre_count_drop = df_high_Year_Genre_count.drop(["All"], axis = 0)

#장르별 점유율 계산하기 (출고량이 높은 게임들 사이에서)
for Plat in df_high_Year_Genre_count_drop.columns:
    df_high_Year_Genre_count_drop[Plat] = df_high_Year_Genre_count_drop[Plat] / df_high_Year_Genre_count_drop["All"]

#필요없는 column 제거
df_high_Year_Genre_count_drop2 = df_high_Year_Genre_count_drop.drop(["All"], axis = 1)

#소수점 자리수 제어
df_high_Year_Genre_ratio = round(df_high_Year_Genre_count_drop2, 2).T

#2010년 이후 데이터만 추림
df_high_Year_Genre_ratio_drop = df_high_Year_Genre_ratio.drop(df_high_Year_Genre_ratio.columns[:29], axis = 1)

#시각화
plt.rcParams["figure.figsize"] = (15, 10)
sns.heatmap(df_high_Year_Genre_ratio_drop,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.3 , #컬러차트 영역
           )

In [ ]:
#출고량이 높은 게임이 전체시장에서 차지하는 비율을 장르별로 나타내기

#출고량이 높은 게임들의 연도-장르별 게임수와 전체 시장에서 연도-장르별 게임수를 DataFrame화 
df_Year_Genre_count = pd.crosstab(df.Year, df.Genre, margins=True)
df_high_Plat_Genre_count = pd.crosstab(df_Sales_high.Platform, df_Sales_high.Genre, margins=True)

#출고량이 높은 게임이 전체 시장에서 차지하는 비율을 계산하기
df_high_Year_Genre_ratio_1 = df_high_Year_Genre_count / df_Year_Genre_count

#2010년 이후 데이터만 추림
df_high_Year_Genre_ratio2 = df_high_Year_Genre_ratio_1.T.drop(df_high_Year_Genre_ratio_1.T.columns[:30], axis = 1)

#필요없는 row, column 제거
df_high_Year_Genre_ratio2.drop(["All"], axis = 1, inplace = True)
df_high_Year_Genre_ratio2.drop(["All"], axis = 0, inplace = True)

#결측치 처리
df_high_Year_Genre_ratio2.fillna(0)

#시각화
plt.rcParams["figure.figsize"] = (15, 10)
sns.heatmap(df_high_Year_Genre_ratio2,
           annot = True, #실제 값 화면에 나타내기
           cmap = 'Greens', #색상
           vmin = 0, vmax = 0.3 , #컬러차트 영역
           )